In [1]:
import os
import pandas as pd
import folium
import numpy as np
import requests
import io
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from folium.plugins import MarkerCluster
from folium import LayerControl

In [2]:
countries = ['Angola', 'Burundi', 'Cameroon', 'Central African Republic', 'Chad', 'Republic of the Congo', 'DR Congo', 'Equatorial Guinea', 'Gabon']

base_dir = ''
folder_in = os.path.join(base_dir, 'input')
folder_out = os.path.join(base_dir, 'output')
if not os.path.exists(folder_out):
    os.makedirs(folder_out)

In [3]:
file = os.path.join(folder_in, 'Global-Integrated-Power-April-2025.xlsx')

## Data description
- Capacity (MW)
- Status
- Technology
- Latitude/Longitude

In [4]:
if os.path.exists(file):
    df = pd.read_excel(file, sheet_name='Power facilities', header=[0], index_col=None)
else:
    raise FileNotFoundError(f"File {file} does not exist.")

df = df[df['Country/area'].isin(countries)]

/opt/anaconda3/envs/HydroData/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [5]:
# Function to fetch and process population data
def get_population_data(resolution='low'):
    """
    Fetch population data from WorldPop at the specified resolution.

    Parameters:
    resolution (str): 'low', 'medium', or 'high'

    Returns:
    tuple: (data, bounds, cmap) where data is the population density array,
           bounds are the geographical bounds, and cmap is the colormap
    """
    # WorldPop global population data (2020) at 1km resolution
    # Using the constrained individual countries 2020 dataset (100m resolution)
    # For low resolution, we'll use the global dataset

    try:
        print("Fetching population data...")

        # For demonstration, we'll use a sample of the WorldPop data
        # In a production environment, you would download the actual data files

        # Simulated population data for Central Africa
        # This is a placeholder - in a real implementation, you would fetch actual data
        import numpy as np

        # Create a simple population density grid (just for demonstration)
        # In reality, you would load this from WorldPop or another source
        lat_range = np.linspace(-15, 15, 100)  # Approximate latitude range for Central Africa
        lon_range = np.linspace(5, 35, 100)    # Approximate longitude range for Central Africa

        # Create a grid of coordinates
        lon_grid, lat_grid = np.meshgrid(lon_range, lat_range)

        # Simulate population density (higher near the equator and certain longitudes)
        # This is just a placeholder pattern
        population = np.exp(-((lat_grid/10)**2)) * np.exp(-((lon_grid-20)/10)**2) * 1000

        # Add some random variation
        population += np.random.normal(0, 0.1, population.shape) * population
        population = np.maximum(population, 0)  # Ensure no negative values

        # Define bounds
        bounds = [lon_range.min(), lat_range.min(), lon_range.max(), lat_range.max()]

        # Define a colormap for population density
        import matplotlib.cm as cm
        cmap = cm.YlOrRd

        print("Population data ready.")
        return population, bounds, cmap, lat_range, lon_range

    except Exception as e:
        print(f"Error fetching population data: {e}")
        return None, None, None, None, None

In [6]:
# Create a folium map centered on
center_lat = df["Latitude"].mean()
center_lon = df["Longitude"].mean()

# Create the base map
power_map = folium.Map(location=[center_lat, center_lon], zoom_start=4)

# Create feature groups for different layers
# Main layer for all power plants
power_plants_layer = folium.FeatureGroup(name="All Power Plants")

# Create separate feature groups for each status
status_layers = {}
status_colors = {
    'Operating': 'green',
    'Under Construction': 'orange',
    'Planned': 'blue',
    'Announced': 'purple',
    'Mothballed': 'gray',
    'Cancelled': 'red',
    'Retired': 'black'
}
for status, color in status_colors.items():
    status_layers[status] = folium.FeatureGroup(name=f"Status: {status}")

# Create separate feature groups for each technology
tech_layers = {}
tech_icons = {
    'Hydro': 'tint',
    'Solar': 'sun',
    'Wind': 'wind',
    'Thermal': 'fire',
    'Gas': 'gas-pump',
    'Coal': 'industry',
    'Oil': 'oil-can',
    'Nuclear': 'atom',
    'Geothermal': 'temperature-high',
    'Biomass': 'leaf'
}
for tech, icon in tech_icons.items():
    tech_layers[tech] = folium.FeatureGroup(name=f"Technology: {tech}")

# Population density layer
population_layer = folium.FeatureGroup(name="Population Density")

# Add population data layer (optional)
population_data, bounds, cmap, lat_range, lon_range = get_population_data(resolution='low')
population_data = None
if population_data is not None:
    # Create a heatmap-like visualization for population density
    # Convert the population data to a format suitable for folium
    population_points = []

    # Sample the data to reduce the number of points (for performance)
    sample_rate = 2  # Use every 2nd point

    max_population = population_data.max()

    for i in range(0, len(lat_range), sample_rate):
        for j in range(0, len(lon_range), sample_rate):
            # Only include points with significant population
            if population_data[i, j] > max_population * 0.01:  # Filter out very low values
                # [lat, lon, intensity]
                population_points.append([
                    lat_range[i], 
                    lon_range[j], 
                    float(population_data[i, j]) / max_population  # Normalize to 0-1
                ])

    # Add the heatmap layer
    from folium.plugins import HeatMap
    HeatMap(
        population_points,
        radius=15,  # Adjust for desired resolution
        blur=10,
        gradient={0.4: 'blue', 0.65: 'lime', 0.8: 'yellow', 1: 'red'},
        max_zoom=10,
        name="Population Density"
    ).add_to(population_layer)

    # Add the population layer to the map
    population_layer.add_to(power_map)

# Create a JavaScript function to create custom cluster icons based on total capacity and dominant technology
cluster_js = """
function(cluster) {
    // Get all markers in the cluster
    var markers = cluster.getAllChildMarkers();
    var totalCapacity = 0;
    var techCounts = {};
    var dominantTech = '';
    var maxCount = 0;

    // Calculate total capacity and count technologies
    markers.forEach(function(marker) {
        // Get capacity from marker options
        if (marker.options.capacity) {
            totalCapacity += marker.options.capacity;
        }

        // Count technology occurrences
        if (marker.options.technology) {
            var tech = marker.options.technology;
            if (!techCounts[tech]) {
                techCounts[tech] = 0;
            }
            techCounts[tech]++;

            // Track the dominant technology
            if (techCounts[tech] > maxCount) {
                maxCount = techCounts[tech];
                dominantTech = tech;
            }
        }
    });

    // Scale the icon size based on total capacity (using square root for better scaling)
    var size = Math.sqrt(totalCapacity) * 1.5;
    if (size < 20) size = 20; // Minimum size for visibility

    // Get the appropriate icon for the dominant technology
    var techIcon = 'bolt'; // Default icon
    var techMapping = {
        'Hydro': 'tint',
        'Solar': 'sun',
        'Wind': 'wind',
        'Thermal': 'fire',
        'Gas': 'gas-pump',
        'Coal': 'industry',
        'Oil': 'oil-can',
        'Nuclear': 'atom',
        'Geothermal': 'temperature-high',
        'Biomass': 'leaf'
    };

    // Find the icon for the dominant technology
    for (var tech in techMapping) {
        if (dominantTech.toLowerCase().includes(tech.toLowerCase())) {
            techIcon = techMapping[tech];
            break;
        }
    }

    // Create the HTML for the cluster icon
    return L.divIcon({
        html: '<div style="background-color: #3388ff; color: white; border-radius: 50%; text-align: center; width: ' + size + 'px; height: ' + size + 'px; line-height: ' + size + 'px; font-size: ' + (size/2) + 'px;"><i class="fa fa-' + techIcon + '"></i></div>',
        className: 'marker-cluster',
        iconSize: L.point(size, size)
    });
}
"""

# Create marker clusters for each layer with custom icon function
main_marker_cluster = MarkerCluster(icon_create_function=cluster_js).add_to(power_plants_layer)
status_marker_clusters = {status: MarkerCluster(icon_create_function=cluster_js).add_to(layer) for status, layer in status_layers.items()}
tech_marker_clusters = {tech: MarkerCluster(icon_create_function=cluster_js).add_to(layer) for tech, layer in tech_layers.items()}

# Add all layers to the map
power_plants_layer.add_to(power_map)
for status, layer in status_layers.items():
    layer.add_to(power_map)
for tech, layer in tech_layers.items():
    layer.add_to(power_map)

# Define technology to icon mapping
def get_icon_for_technology(technology):
    tech_icons = {
        'Hydro': 'tint',  # water drop
        'Solar': 'sun',
        'Wind': 'wind',
        'Thermal': 'fire',
        'Gas': 'gas-pump',
        'Coal': 'industry',
        'Oil': 'oil-can',
        'Nuclear': 'atom',
        'Geothermal': 'temperature-high',
        'Biomass': 'leaf'
    }

    # Default icon for unknown technologies
    default_icon = 'bolt'  # electricity bolt

    # Check if the technology contains any of the keys
    for key, icon in tech_icons.items():
        if key.lower() in str(technology).lower():
            return icon

    return default_icon

# Define status to color mapping
def get_color_for_status(status):
    status_colors = {
        'Operating': 'green',
        'Under Construction': 'orange',
        'Planned': 'blue',
        'Announced': 'purple',
        'Mothballed': 'gray',
        'Cancelled': 'red',
        'Retired': 'black'
    }

    # Default color for unknown status
    default_color = 'cadetblue'

    # Check if the status contains any of the keys
    for key, color in status_colors.items():
        if key.lower() in str(status).lower():
            return color

    return default_color

# Function to scale capacity for marker size (using square root to make scaling more reasonable)
def scale_capacity(capacity):
    if pd.isna(capacity) or capacity <= 0:
        return 5  # Default size for unknown or zero capacity
    return np.sqrt(capacity) * 1.5  # Scale factor can be adjusted

# Add markers for each power plant
for idx, row in df.iterrows():
    # Skip if latitude or longitude is missing
    if pd.isna(row['Latitude']) or pd.isna(row['Longitude']):
        continue

    # Get marker properties
    capacity = row.get('Capacity (MW)', 0)
    technology = row.get('Technology', 'Unknown')
    status = row.get('Status', 'Unknown')

    # Create a more comprehensive popup content with all available information
    popup_content = f"""
    <b>Name:</b> {row.get('Name', 'Unknown')}<br>
    <b>Technology:</b> {technology}<br>
    <b>Capacity:</b> {capacity} MW<br>
    <b>Status:</b> {status}<br>
    <b>Country:</b> {row.get('Country/area', 'Unknown')}<br>
    """

    # Add any other available columns to the popup
    for col in row.index:
        if col not in ['Name', 'Technology', 'Capacity (MW)', 'Status', 'Country/area', 'Latitude', 'Longitude'] and not pd.isna(row[col]):
            popup_content += f"<b>{col}:</b> {row[col]}<br>"

    # Create a single marker with icon representing technology
    # The icon color represents status
    # We'll use DivIcon to create a custom marker that shows capacity by size
    icon_size = scale_capacity(capacity)

    # Create a custom icon that combines technology icon with capacity representation
    # Use DivIcon to create a marker with size based on capacity

    # Create HTML for the custom icon with size based on capacity
    icon_html = f"""
    <div style="
        background-color: {get_color_for_status(status)};
        color: white;
        border-radius: 50%;
        text-align: center;
        width: {icon_size*2}px;
        height: {icon_size*2}px;
        line-height: {icon_size*2}px;
        font-size: {icon_size}px;
    ">
        <i class="fa fa-{get_icon_for_technology(technology)}"></i>
    </div>
    """

    # Create marker with capacity and technology stored in options for cluster calculations
    marker = folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        icon=folium.DivIcon(
            icon_size=(icon_size*2, icon_size*2),
            icon_anchor=(icon_size, icon_size),
            html=icon_html
        ),
        popup=folium.Popup(popup_content, max_width=300),
        tooltip=f"{row.get('Name', 'Unknown')} - {technology} - {capacity} MW"
    )

    # Store capacity and technology in marker options for cluster calculations
    marker.options['capacity'] = float(capacity) if not pd.isna(capacity) else 0
    marker.options['technology'] = str(technology)

    # Add marker to the main layer
    marker.add_to(main_marker_cluster)

    # Add marker to the appropriate status layer
    for status_name, cluster in status_marker_clusters.items():
        if status_name.lower() in str(status).lower():
            # Create a copy of the marker with the same custom DivIcon
            marker_copy = folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                icon=folium.DivIcon(
                    icon_size=(icon_size*2, icon_size*2),
                    icon_anchor=(icon_size, icon_size),
                    html=icon_html
                ),
                popup=folium.Popup(popup_content, max_width=300),
                tooltip=f"{row.get('Name', 'Unknown')} - {technology} - {capacity} MW"
            )
            marker_copy.add_to(cluster)
            break

    # Add marker to the appropriate technology layer
    for tech_name, cluster in tech_marker_clusters.items():
        if tech_name.lower() in str(technology).lower():
            # Create a copy of the marker with the same custom DivIcon
            marker_copy = folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                icon=folium.DivIcon(
                    icon_size=(icon_size*2, icon_size*2),
                    icon_anchor=(icon_size, icon_size),
                    html=icon_html
                ),
                popup=folium.Popup(popup_content, max_width=300),
                tooltip=f"{row.get('Name', 'Unknown')} - {technology} - {capacity} MW"
            )
            marker_copy.add_to(cluster)
            break

# Add a legend for status colors
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 180px; height: auto; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; padding: 10px;
            border-radius: 6px;">
    <p style="margin-top: 0; margin-bottom: 5px;"><b>Status</b></p>
'''

# Add each status color to the legend
for status, color in [
    ('Operating', 'green'),
    ('Under Construction', 'orange'),
    ('Planned', 'blue'),
    ('Announced', 'purple'),
    ('Mothballed', 'gray'),
    ('Cancelled', 'red'),
    ('Retired', 'black')
]:
    legend_html += f'''
    <div style="display: flex; align-items: center; margin-bottom: 3px;">
        <div style="background-color:{color}; width:15px; height:15px; margin-right:5px; border-radius:50%;"></div>
        <span>{status}</span>
    </div>
    '''


legend_html += '''
    <p style="margin-top: 10px; margin-bottom: 5px;"><b>Size</b></p>
    <div>Marker size is proportional to Capacity (MW)</div>
    <div>Icon background color represents Status</div>
    <div>Tooltip shows Name, Technology, and Capacity</div>
'''

# Update the legend to include population information
legend_html += '''
    <p style="margin-top: 10px; margin-bottom: 5px;"><b>Population Density</b></p>
    <div style="display: flex; align-items: center; margin-bottom: 3px;">
        <div style="background: linear-gradient(to right, blue, lime, yellow, red); width:100px; height:10px; margin-right:5px;"></div>
    </div>
    <div>Low to High Density</div>
    <div style="font-size: 12px; margin-top: 3px;">(Toggle layer visibility in control panel)</div>

    <p style="margin-top: 10px; margin-bottom: 5px;"><b>Filtering Options</b></p>
    <div style="font-size: 12px;">Use the layer control panel in the top-right corner to:</div>
    <div style="font-size: 12px;">- Show all power plants</div>
    <div style="font-size: 12px;">- Filter by Status (e.g., Operating, Planned)</div>
    <div style="font-size: 12px;">- Filter by Technology (e.g., Hydro, Solar)</div>
</div>
'''

# Add the legend to the map
power_map.get_root().html.add_child(folium.Element(legend_html))

# Add layer control to toggle layers
folium.LayerControl().add_to(power_map)

# Display the map
power_map


# Save the map to an HTML file
output_file = os.path.join(folder_out, 'power_map.html')
power_map.save(output_file)


Fetching population data...
Population data ready.
